In [1]:
import pymysql
import math
import scipy.stats as stat
from scipy.stats import norm
import pandas as pd
import numpy as np
import urllib.request, json
import sqlalchemy

In [25]:
#grab the entire table from sql
#contains recent traded option log and updated gex/vex/ddoi values
#make this take an argument in case we have more tables in the future
def get_sql_df():
    Host = 'deribit-db-instance.csduabs5w5go.us-east-1.rds.amazonaws.com'
    User = 'admin'
    Password = 'Deribit123'
    Database = 'deribit'
    #con = pymysql.connect(user=User, password=Password,host=Host, database=Database, port=3306, ssl='../../Downloads/deribit-key-pair.pem')
    #wtf? no ssl param needed anymore?
    con = pymysql.connect(user=User, password=Password,host=Host, database=Database, port=3306)
    try:
        with con.cursor() as cur:
            query = "SELECT *\
                    FROM deribit.option_table\
                    WHERE str_to_date(substring(instrument_name, locate('-',instrument_name)+1,locate('-', instrument_name, locate('-',instrument_name)+1) - locate('-',instrument_name)-1), '%d%b%y') >= CURDATE();"
            cur.execute(query)
            df = pd.read_sql(query, con)
            return df
    finally:
        con.close()
        

In [26]:
df = get_sql_df()
#df

In [27]:
df.shape

(56, 15)

In [28]:
#append last trade as a new trade to simulate what we would do if we just got a new trade
new_trade = df.iloc[-1].copy()
df.loc[(df.shape[0] + 1)] = new_trade
df.shape

(57, 15)

In [29]:
testingdf = df.copy()
#testingdf

In [30]:
#testingdf = testingdf.filter(['instrument_name', 'direction', 'amount'])
#testingdf['direction'] = testingdf['direction'].replace('buy', 1)
#testingdf['direction'] = testingdf['direction'].replace('sell', -1)
#testingdf = testingdf.assign(net_amount = lambda x: (x['amount'] * x['direction']))
#aggr_df = testingdf.groupby(['instrument_name'], sort=False).agg({'net_amount': sum})
#aggr_df.reset_index(level=0, inplace=True)

In [31]:
def get_df(df):
    df = df.filter(['instrument_name', 'direction', 'amount'])
    df['direction'] = df['direction'].replace('buy', 1)
    df['direction'] = df['direction'].replace('sell', -1)
    df = df.assign(net_amount = lambda x: (x['amount'] * x['direction']))
    aggr_df = df.groupby(['instrument_name'], sort=False).agg({'net_amount': sum})
    aggr_df.reset_index(level=0, inplace=True)
    return aggr_df

In [32]:
no_stale = get_df(testingdf)

In [34]:
no_stale

,instrument_name,net_amount
0,BTC-26MAR21-54000-C,10
1,BTC-26MAR21-11000-P,0
2,BTC-26MAR21-16000-P,0
3,BTC-26MAR21-10000-P,1
4,BTC-25JUN21-46000-C,38764
5,BTC-25JUN21-60000-C,38765
6,BTC-26MAR21-28000-C,38


In [35]:
get_orderbook_url = "https://test.deribit.com/api/v2/public/get_order_book?depth=5&instrument_name=BTC-5MAR21-57000-C"
with urllib.request.urlopen(get_orderbook_url) as get_orderbook_url:
    ob_data = json.loads(get_orderbook_url.read().decode())
ob_data = pd.DataFrame.from_dict(ob_data['result'], orient='index')
ob_data = ob_data.transpose()
ob_data

,underlying_price,underlying_index,timestamp,stats,state,open_interest,min_price,max_price,mark_price,mark_iv,...,delivery_price,change_id,bids,bid_iv,best_bid_price,best_bid_amount,best_ask_price,best_ask_amount,asks,ask_iv
0,47155.7,SYN.BTC-5MAR21,1614931200012,"{'volume': None, 'price_change': None, 'low': ...",closed,0,0.0001,0.2,0,250,...,47191.3,5419243086,[],0,0,0,0,0,[],0


In [36]:
def get_orderbook_data(df_to_append_to, instrument_name):
    #use this to get relevant information from deribit: i.e greeks, open interest,& interest rate
    get_orderbook_url = "https://test.deribit.com/api/v2/public/get_order_book?depth=5&instrument_name="
    get_orderbook_url += instrument_name
    with urllib.request.urlopen(get_orderbook_url) as get_orderbook_url:
        orderbook_data = json.loads(get_orderbook_url.read().decode())
    orderbook_data = pd.DataFrame.from_dict(orderbook_data['result'], orient='index')
    orderbook_data = orderbook_data.transpose()
    combined_df = df_to_append_to.append(orderbook_data, ignore_index=True)
    return combined_df

In [37]:
def get_greeks(active_trades):
    greeks = pd.DataFrame()
    for i in active_trades['instrument_name']:
        greeks = get_orderbook_data(greeks, i)
    columns = greeks.loc[:, ['underlying_price','underlying_index', 'open_interest','interest_rate', 'mark_iv', 'greeks']]
    greeks = pd.DataFrame(columns['greeks'].values.tolist())
    columns = columns.drop(['greeks'], axis = 1)
    greeks = greeks.join(columns)
    greeks['vanna'] = np.nan
    return greeks

In [38]:
greeks = get_greeks(no_stale)

In [39]:
def get_gex(active_trades, active_trades_greeks):
    total_gamma = active_trades['net_amount'] * active_trades_greeks['gamma']
    return sum(total_gamma)

In [40]:
get_gex(no_stale, greeks)

0.7756000000000001

In [50]:
active_trades = no_stale
dte = pd.to_datetime(active_trades['instrument_name'].apply(lambda x: x.split('-')[1]))-pd.Timestamp.today()
strike = active_trades['instrument_name'].apply(lambda x: x.split('-')[2]).astype(int)
active_trades['strike'] = strike
#active_trades.astype({'strike': 'int32'}).dtypes
active_trades['dte'] = dte.dt.days
sigmaT = greeks['mark_iv'] * active_trades['dte'] ** 0.5
print(greeks['underlying_price'])
print(type(math.log(greeks['underlying_price'])/active_trades['strike']))
print(type(active_trades['strike']))
print(type((greeks['interest_rate'])))
print(type(greeks['mark_iv'] ** 2))
print(type(active_trades['dte']))
print(type(sigmaT))
#d1 = (math.log(greeks['underlying_price'] / active_trades['strike']) + \
 #     (greeks['interest_rate'] * 0.5 * (greeks['mark_iv'] ** 2)) \
  #    * active_trades['dte']) / sigmaT

0    57870.3
1    57869.9
2    57869.9
3    57869.9
4    61324.8
5    61324.8
6    57869.8
Name: underlying_price, dtype: object


TypeError: cannot convert the series to <class 'float'>

In [18]:
def get_vanna(active_trades, greeks):
    #get days till expiration from active trades and add as a series
    dte = pd.to_datetime(active_trades['instrument_name'].apply(lambda x: x.split('-')[1]))-pd.Timestamp.today()
    strike = active_trades['instrument_name'].apply(lambda x: x.split('-')[2]).astype(int)
    active_trades['strike'] = strike
    #active_trades.astype({'strike': 'int32'}).dtypes
    active_trades['dte'] = dte.dt.days
    sigmaT = greeks['mark_iv'] * active_trades['dte'] ** 0.5
    d1 = (math.log(greeks['underlying_price'] / active_trades['strike']) + \
                   (greeks['interest_rate'] * 0.5 * (greeks['mark_iv'] ** 2)) \
                   * active_trades['dte']) / sigmaT
    d2 = d1 - sigmaT
    vannas = []
    #we'd like to vectorize this but d1 norm.pdf(d1) is not trivial due to some stupid type error
    #I can't be bothered to figure out right now
    for i in range(active_trades.shape[0]):
        vannas.append(0.01 * -math.e ** active_trades['dte'][i] * d2[i] / greeks['mark_iv'][i] * norm.pdf(d1[i]))
    return vannas
    #[0.01 * -e ** (-self.q * self.T) * self.d2 / self.sigma * norm.pdf(self.d1)]

In [19]:
vanna = get_vanna(no_stale, greeks)

TypeError: cannot convert the series to <class 'float'>

In [ ]:
no_stale

In [ ]:
def get_vex(active_trades, greeks):
    vanna = get_vanna(active_trades, greeks)
    return vanna * no_stale['net_amount']

In [ ]:
vex = get_vex(no_stale, greeks)

In [ ]:
vex

In [ ]:
df

In [ ]:
new_trade

In [ ]:
def post_to_db(new_trade):
    new_trade['gex'] = get_gex(no_stale, greeks)
    new_trade['vex'] = get_vex(no_stale, greeks)
    #in the script, new_trade will be a df instead oa series so this line should
    #be removed
    df_to_post = new_trade.to_frame()
    df_to_post = df_to_post.transpose()
    sqlUrl = sqlalchemy.engine.url.URL(
        drivername="mysql+pymysql",
        username='admin',
        password='Deribit123',
        host='deribit-db-instance.csduabs5w5go.us-east-1.rds.amazonaws.com',
        port=3306,
        database='deribit',
        #make this the server's path and also make it secure?
        query={"ssl_key": "/home/tjang/Downloads/deribit-key-pair.pem"},
    )
    engine = sqlalchemy.create_engine(sqlUrl)
    df_to_post.to_sql('option_table', engine, if_exists='append', index=False)

In [ ]:
post_to_db(new_trade)